# Homework 3

The goal of this homework is to familiarize users with workflow orchestration. We start from the solution of homework 1. The notebook can be found below:

https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/01-intro/homework.ipynb

This has already been converted to a script called `homework.py` in the `03-orchestration` folder of this repo.

You will use the FHV dataset like in homework 1.

## Motivation

We already have a model training script. Maybe a data scientist in your team handed it to you and your job is schedule the running of training script using a workflow orchestration - Prefect in this case. Below are the requirements. Do not implement them yet, we will do so in this exercise. Just understand the goal.

1. The training flow will be run every month.
2. The flow will take in a parameter called `date` which will be a datetime.
   a. `date` should default to `None`
   b. If date is None, set date as the current day.
   Use the data from 2 months back as the training data and the data from the previous month as validation data. c. If date is passed, get 2 months before the date as the training data, and the previous month as validation data. d. As a concrete example, if the `date` passed is "2021-03-15", the training data should be "fhv_tripdata_2021-01.parquet" and the validation file will be "fhv_trip_data_2021-02.parquet"
3. Save the model as "model-{date}.pkl" where date is in `YYYY-MM-DD`. Note that `date` here is the value of the flow `parameter`. In practice, this setup makes it very easy to get the latest model to run predictions because you just need to get the most recent one.
4. In this example we use a DictVectorizer. That is needed to run future data through our model. Save that as "dv-{date}.pkl". Similar to above, if the date is `2021-03-15`, the files output should be `model-2021-03-15.bin` and `dv-2021-03-15.b`.

This convention is not strict in industry, and in practice, you will come up with your own system to manage these training pipeline runs. For example, if we wanted to train on the whole history instead of just one month, we'd need to allow for added parameterization and logic in our flow. If the data came in weekly instead of monthly, we might need a different naming convention. But these requirements are already a simple approximation of something you could use in production.

On the deployment side, it's very easy to just pull in the latest data and predict it using the latest model and vectorizer files. Tools the MLFlow in the last chapter can simplify that process as well. This homework will focus more on the batch training.

In order, this homework assignment will be about:

1. Converting the script to a Flow
2. Changing the parameters to take in a `date`. Making this parameter dynamic.
3. Scheduling a batch training job that outputs the latest model somewhere

## Setup

You can use either local Prefect Orion or a VM hosted Prefect Orion instance for this. It shouldn't matter. Just note that if you use a VM hosted one, you will need to configure your local API to hit the VM.

Video 3.4 of the course will give more detailed instructions if you been run it on a VM.

## Q1. Converting the script to a Prefect flow

If you want to follow the videos exactly, do:

```
pip install prefect==2.0b5
```

If you need Windows support, check `windows.md` for installation instructions.

The current script `homework.py` is a fully functional script as long as you already have `fhv_trip_data_2021-01.parquet` and `fhv_trip_data_2021-02.parquet` inside a `data` folder. You should be able to already run it using:

```
python homework.py
```

We want to bring this to workflow orchestration to add observability around it. The `main` function will be converted to a `flow` and the other functions will be tasks. After adding all of the decorators, there is actually one `task` that you will need to call `.result()` for inside the `flow` to get it to work. Which task is this?

* `read_data`
* `prepare_features`
* `train_model`
* `run_model`

Important: change all print statements to use the Prefect logger. Using the `print` statement will not appear in the Prefect UI. You have to call `get_run_logger` at the start of the task to use it.

Answer: `train_model`

[Using resutls from tasks][task-results] in Prefect's documentation states that tasks returns [`PrefectFuture`][prefectfuture]. When the returned value of one task is given directly to another task, the receiving task will "unwrap" the given `PrefectFuture` before running the function. However, `run_model` destructures `train_model`'s in `@flow main`, explicitly calling `.result()` is necessary. The code in question is as follows:

```python
@flow(task_runner=SequentialTaskRunner())
def main(train_path: str = './data/fhv_tripdata_2021-01.parquet',
           val_path: str = './data/fhv_tripdata_2021-02.parquet'):

    categorical = ['PUlocationID', 'DOlocationID']

    df_train = read_data(train_path)
    df_train_processed = prepare_features(df_train, categorical)     # no need for df_train.result(), 'cos `prepare_feature` task will "unwrap" it automatically

    df_val = read_data(val_path)
    df_val_processed = prepare_features(df_val, categorical, False)    
    # train the model
    lr, dv = train_model(df_train_processed, categorical).result()   # destructuring returned value here, thus calling .result() is necessary
    run_model(df_val_processed, categorical, dv, lr)
```

[task-results]: https://orion-docs.prefect.io/concepts/tasks/#using-results-from-tasks
[prefectfuture]: https://orion-docs.prefect.io/api-ref/prefect/futures/#prefect.futures.PrefectFuture